In [3]:
from mysql.connector import MySQLConnection, Error

In [6]:
def make_connection():
    """
    Make a connection to the school database.
    """
    try:
        conn = MySQLConnection(host = 'IES-ADS-ClassDB.sjsu.edu',
                               database = 'querycrew_wh',
                               user = 'querycrew_user',
                               password = 'Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [8]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


In [14]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget,
    QVBoxLayout, QHBoxLayout, QPushButton,
    QLabel, QInputDialog, QLineEdit, QMessageBox,
    QTableView, QHeaderView, QComboBox, QStyle
)
from PyQt5.QtGui import QPainter, QBrush, QColor
from PyQt5.QtCore import Qt
from PyQt5.QtSql import QSqlDatabase, QSqlTableModel, QSqlQueryModel, QSqlQuery

class CarGraphic(QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setFixedSize(80, 40)

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setBrush(QBrush(QColor('#3498db')))
        painter.drawRect(10, 15, 60, 15)
        painter.drawRect(20, 5, 40, 15)
        painter.setBrush(QBrush(QColor('#2c3e50')))
        painter.drawEllipse(15, 28, 14, 14)
        painter.drawEllipse(51, 28, 14, 14)
        painter.end()

class OperationalApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("CarSights Main Menu")
        self.setFixedSize(600, 330)
        self.db = self.init_db()
        self.init_menu()

    def init_db(self):
        db = QSqlDatabase.addDatabase("QMYSQL")
        db.setHostName("IES-ADS-ClassDB.sjsu.edu")
        db.setDatabaseName("querycrew_wh")
        db.setUserName("querycrew_user")
        db.setPassword("Pomegranate_746")
        if not db.open():
            QMessageBox.critical(None, "DB Connection Error", db.lastError().text())
            sys.exit(1)
        return db

    def init_menu(self):
        menu = QWidget()
        layout = QVBoxLayout(menu)
        layout.setSpacing(5)
        layout.setContentsMargins(15,15,15,15)

        layout.addWidget(CarGraphic(), alignment=Qt.AlignCenter)
        greeting = QLabel("Hello, welcome to CarSights!")
        greeting.setAlignment(Qt.AlignCenter)
        greeting.setStyleSheet("font-size:24px; font-weight:bold;")
        layout.addWidget(greeting)

        btn_layout = QHBoxLayout()
        btn_layout.setSpacing(20)
        btn_cust = QPushButton("Customers Portal")
        btn_cust.setIcon(self.style().standardIcon(QStyle.SP_FileIcon))
        btn_cust.clicked.connect(self.show_customers_portal)
        btn_layout.addWidget(btn_cust)
        btn_deal = QPushButton("Dealers Portal")
        btn_deal.setIcon(self.style().standardIcon(QStyle.SP_DesktopIcon))
        btn_deal.clicked.connect(self.request_dealer_password)
        btn_layout.addWidget(btn_deal)
        layout.addLayout(btn_layout)

        self.setCentralWidget(menu)

    def request_dealer_password(self):
        pwd, ok = QInputDialog.getText(self, "Password Required", "Enter dealer portal password:", QLineEdit.Password)
        if ok and pwd == "admin123":
            self.show_dealers_portal()
        else:
            QMessageBox.warning(self, "Access Denied", "Incorrect password.")

    def show_dealers_portal(self):
        portal = QWidget()
        layout = QVBoxLayout(portal)
        layout.setSpacing(5)
        layout.setContentsMargins(15,15,15,15)
        title = QLabel("Welcome to the Dealers Portal!")
        title.setAlignment(Qt.AlignCenter)
        title.setStyleSheet("font-weight:bold; font-size:18px;")
        layout.addWidget(title)
        btn_view = QPushButton("View Customers")
        btn_view.setIcon(self.style().standardIcon(QStyle.SP_DirIcon))
        btn_view.clicked.connect(lambda: self.show_table("Customers", self.show_dealers_portal))
        layout.addWidget(btn_view)
        btn_monthly = QPushButton("Monthly EV Sales")
        btn_monthly.setIcon(self.style().standardIcon(QStyle.SP_DriveHDIcon))
        btn_monthly.clicked.connect(self.function4)
        layout.addWidget(btn_monthly)
        btn_back = QPushButton("Back to Main Menu")
        btn_back.clicked.connect(self.init_menu)
        layout.addWidget(btn_back)
        self.setCentralWidget(portal)

    def show_customers_portal(self):
        portal = QWidget()
        layout = QVBoxLayout(portal)
        layout.setSpacing(5)
        layout.setContentsMargins(15,15,15,15)
        title = QLabel("Welcome to the Customers Portal!")
        title.setAlignment(Qt.AlignCenter)
        title.setStyleSheet("font-weight:bold; font-size:18px;")
        layout.addWidget(title)
        btn_avg = QPushButton("Average Car Cost")
        btn_avg.setIcon(self.style().standardIcon(QStyle.SP_ArrowUp))
        btn_avg.clicked.connect(self.function2)
        layout.addWidget(btn_avg)
        btn_search = QPushButton("Search for a car")
        btn_search.setIcon(self.style().standardIcon(QStyle.SP_FileDialogDetailedView))
        btn_search.clicked.connect(self.function3)
        layout.addWidget(btn_search)
        btn_back = QPushButton("Back to Main Menu")
        btn_back.clicked.connect(self.init_menu)
        layout.addWidget(btn_back)
        self.setCentralWidget(portal)

    def function2(self):
        model = QSqlQueryModel(self)
        model.setQuery("SELECT car_make, car_model, CONCAT('$',ROUND(AVG(price),2)) AS avg_price FROM car_sales GROUP BY car_make, car_model", self.db)
        self.show_model_view(model, back_callback=self.show_customers_portal)

    def function3(self):
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.setSpacing(2)
        layout.setContentsMargins(5,5,5,5)
        row1 = QHBoxLayout()
        row1.setContentsMargins(0,0,0,0)
        row1.setSpacing(5)
        row1.addWidget(QLabel("Select car make:"))
        self.make_combo = QComboBox()
        row1.addWidget(self.make_combo)
        layout.addLayout(row1)
        q = QSqlQuery(self.db)
        q.exec_("SELECT DISTINCT car_make FROM Car ORDER BY car_make")
        while q.next(): self.make_combo.addItem(q.value(0))
        self.make_combo.currentTextChanged.connect(self.populate_models)
        row2 = QHBoxLayout()
        row2.setContentsMargins(0,0,0,0)
        row2.setSpacing(5)
        row2.addWidget(QLabel("Select car model:"))
        self.model_combo = QComboBox()
        row2.addWidget(self.model_combo)
        layout.addLayout(row2)
        self.populate_models(self.make_combo.currentText())
        btn_inv = QPushButton("Search Inventory")
        btn_inv.setMaximumHeight(24)
        btn_inv.clicked.connect(self.run_inventory_search)
        layout.addWidget(btn_inv)
        back_btn = QPushButton("Back to Customers Portal")
        back_btn.setMaximumHeight(24)
        back_btn.clicked.connect(self.show_customers_portal)
        layout.addWidget(back_btn)
        self.setCentralWidget(widget)

    def populate_models(self, make):
        self.model_combo.clear()
        q = QSqlQuery(self.db)
        q.prepare("SELECT DISTINCT car_model FROM Car WHERE car_make=? ORDER BY car_model")
        q.addBindValue(make)
        q.exec_()
        while q.next(): self.model_combo.addItem(q.value(0))

    def run_inventory_search(self):
        make = self.make_combo.currentText()
        model = self.model_combo.currentText()
        sql = (f"SELECT c.car_id,c.car_make,c.car_model,c.year,c.car_cost,d.dealer_name "
               f"FROM Car c JOIN dealer_sales_summary d ON c.dealer_id=d.dealer_id "
               f"WHERE c.car_make='{make}' AND c.car_model='{model}' "
               "ORDER BY c.year DESC")
        inv_model = QSqlQueryModel(self)
        inv_model.setQuery(sql, self.db)
        self.show_model_view(inv_model, back_callback=self.show_customers_portal)

    def function4(self):
        model = QSqlQueryModel(self)
        model.setQuery(
            "SELECT dealer_id, dealer_name, COUNT(*) AS ev_sales "
            "FROM EV_arrivals JOIN dealer_sales_summary USING(dealer_id) "
            "GROUP BY dealer_id, dealer_name",
            self.db
        )
        self.show_model_view(model, back_callback=self.show_dealers_portal)

    def show_table(self, table, back_callback):
        model = QSqlTableModel(self, self.db)
        model.setTable(table)
        model.select()
        self.show_model_view(model, back_callback)

    def show_model_view(self, model, back_callback):
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.setSpacing(5)
        layout.setContentsMargins(10,10,10,10)
        view = QTableView()
        view.setModel(model)
        view.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        layout.addWidget(view)
        btn_back = QPushButton("Back")
        btn_back.clicked.connect(back_callback)
        layout.addWidget(btn_back)
        self.setCentralWidget(widget)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = OperationalApp()
    window.show()
    sys.exit(app.exec_())


SystemExit: 0

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
cursor.close()
conn.close()